In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import mne
from sklearn.preprocessing import LabelEncoder
from scipy.fft import fft


## Análise  da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características. (2) seleção de características e (3) classificação dos dados.

### Pontos Importantes do Dataset
 
- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz


### Análisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados do participante.
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz.
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo).
   - **a)** 0.0 - 0.5 segundos e  2.5 - 3.0 segundos ocorre apenas ruído;
   - **b**) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos MNE, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações. 
   - Os dados devem ser plotados no dominio da frequência (após a transformada de Fourier). O FFT pode ser realizada pela biblioteca `scipy.fft`.
   - Deve ser observado que as janelas (a) com ruídos não aparecerão de fato o sinal SSVEP.

In [9]:
data = np.load("../../datasets/beta/data.npy")

print(data.shape)


(160, 64, 750)


In [10]:
n_channels = 64
sfreq = 512
ch_names = list(np.load("../../datasets/beta/channels.npy"))
ch_types = ['eeg'] * len(ch_names)
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
best = ['P6', 'PO3']

In [11]:
labels = np.load("../../datasets/beta/labels.npy")
unique_labels = sorted(set(labels))
event_dict = {str(value): index  for index, value in enumerate(unique_labels)}
labels.shape

(160,)

In [19]:
# método para transformar labels categóricos
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)

# print(events.shape)

mne_data = mne.EpochsArray(data, info, events, event_id=event_dict)
filtered_mne_data = mne_data.filter(6, 18)
filtered_mne_data

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 6 - 18 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 6.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Upper passband edge: 18.00 Hz
- Upper transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 20.25 Hz)
- Filter length: 845 samples (1.650 s)



/tmp/ipykernel_7374/2045036464.py:12: RuntimeWarning: filter_length (845) is longer than the signal (750), distortion is likely. Reduce filter length or filter a longer signal.
  filtered_mne_data = mne_data.filter(6, 18)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 3041 tasks    

Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 1.463 s
Baseline,off


In [17]:
start_noise = filtered_mne_data.copy().crop(tmin=0.0, tmax=0.5)
middle = filtered_mne_data.copy().crop(tmin=0.5, tmax=2.5)
end_noise = filtered_mne_data.copy().crop(tmin=2.5, tmax=3.0)

# 4. Aplicar a FFT para plotar gráficos no domínio da frequência.
def plot_fft(data, title):
    times = data.times
    signal = data.get_data()[0]  # Assumindo que estamos interessados no primeiro canal
    n_samples = len(signal)
    sample_rate = data.info['sfreq']
    frequencies = np.fft.fftfreq(n_samples, 1/sample_rate)
    fft_values = fft(signal)
    fft_magnitude = np.abs(fft_values)
    
    plt.figure()
    plt.plot(frequencies, fft_magnitude)
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('Magnitude da FFT')
    plt.title(title)
    plt.show()

# Plotar FFT para as duas janelas de interesse
plot_fft(start_noise, 'FFT - Janela a (0.0 - 0.5 segundos)')
plot_fft(middle, 'FFT - Janela b (0.5 - 2.5 segundos)')
plot_fft(end_noise, 'FFT - Janela b (2.5 - 3.5 segundos)')

/tmp/ipykernel_7374/654663683.py:2: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (1.46289 s)
  middle = filtered_mne_data.copy().crop(tmin=0.5, tmax=2.5)
/tmp/ipykernel_7374/654663683.py:3: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (1.46289 s)
  end_noise = filtered_mne_data.copy().crop(tmin=2.5, tmax=3.0)


ValueError: tmin (2.5) must be less than or equal to tmax (1.462890625)

#### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `SNR` (obrigatória);
- Maior valor espectral (FFT);
- Médio dos valores espectrais (FFT);


Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 channels e 750 values.

`160, 64 (SRN) + 64 (MÉDIA) + 64 (MAIOR) ...`

Resultando em  `160, 192`

#### Seleção de características e classsificação

Como existem diversos eletrodos (canais) que não obtem sinal SSVEP, podemos extrair as características que não contribuem para a classificação dos dados.


Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicador por meio de `sklearn.feature_selection.RFE`, aprimorand o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.

# How to calculate SNR https://saturncloud.io/blog/calculating-signaltonoise-ratio-in-python-with-scipy-v11/